<a href="https://colab.research.google.com/github/albivaltzew/urbancode_samolet/blob/main/albumentations_coco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [27]:
!pip install --upgrade albumentations

In [18]:
import os
import cv2
import json
import albumentations as A
import numpy as np

In [21]:
coco_file = "/content/gdrive/MyDrive/urbanhack-train/annotations/instances_default.json"

# Specify the folder path where your images are located
image_folder = '/content/gdrive/MyDrive/urbanhack-train/images'  # Replace with the actual path to your image folder


In [8]:
# Load your COCO-style annotation file
with open(coco_file, 'r') as f:
    coco_data = json.load(f)

In [9]:
# Load images and corresponding annotations
images = coco_data['images']
annotations = coco_data['annotations']

In [11]:
# Define an augmentation pipeline
augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=45, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
])

In [12]:
augmented_images = []
augmented_annotations = []

In [ ]:
for image_info in images:

    image_path = os.path.join(image_folder, image_info['file_name'])

    print(image_path)
    image = cv2.imread(image_path)
    image = np.array(image)

    print(f"Loaded image: {image_info['file_name']}, shape: {image.shape}")  # Debugging

    augmented = augmentation(image=image)

    augmented_image = augmented['image']


    # Transform bounding box coordinates
    transformed_annotations = []
    for annotation in annotations:
        if annotation['image_id'] == image_info['id']:
            bbox = annotation['bbox']
            x, y, w, h = bbox

            # Apply augmentation to bounding box
            transformed_bbox = A.core.bbox_utils.normalize_bbox(
                [x, y, x + w, y + h], rows=image.shape[0], cols=image.shape[1]
            )
            transformed_bbox = A.core.bbox_utils.denormalize_bbox(
                transformed_bbox, rows=augmented_image.shape[0], cols=augmented_image.shape[1]
            )

            transformed_annotations.append({
                'id': annotation['id'],
                'image_id': image_info['id'],
                'category_id': annotation['category_id'],
                'bbox': transformed_bbox,
                'iscrowd': annotation['iscrowd'],
                'attributes': annotation['attributes'],  # You may need to adjust this if needed
            })

    augmented_images.append(augmented_image)
    augmented_annotations.extend(transformed_annotations)

In [35]:
# Save the augmented dataset
augmented_coco_data = {
    'images': images,
    'annotations': augmented_annotations,
    'categories': coco_data['categories'],  # Ensure 'categories' is present in your original JSON
}

In [37]:
augmented_path = "/content/gdrive/MyDrive/urbanhack-train/augmented_annotations.json"

In [38]:
with open(augmented_path, 'w') as f:
    json.dump(augmented_coco_data, f)

In [39]:
# Папка для сохранения аугментированных изображений
output_folder = '/content/gdrive/MyDrive/urbanhack-train/augmented_images'  # Замените на путь к папке, в которой хотите сохранить аугментированные изображения
os.makedirs(output_folder, exist_ok=True)

In [40]:
for i, augmented_image in enumerate(augmented_images):
    # Генерируйте уникальное имя файла для каждого изображения
    output_filename = os.path.join(output_folder, f'augmented_image_{i}.jpg')

    # Сохраните аугментированное изображение
    cv2.imwrite(output_filename, augmented_image)

Подсчет количества файлов в папке

In [41]:
# Укажите путь к папке, в которой вы хотите посчитать файлы
folder_path = '/content/gdrive/MyDrive/urbanhack-train/augmented_images'

# Инициализируем счетчик файлов
file_count = 0

# Перебираем файлы в папке
for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)):
        file_count += 1

print(f"Количество файлов в папке: {file_count}")

Количество файлов в папке: 252


BBOXParams

In [53]:
image_folder = "/content/gdrive/MyDrive/urbanhack-train/images"

# Load the original COCO dataset
original_coco_file = '/content/gdrive/MyDrive/urbanhack-train/annotations/instances_default.json'  # Replace with the path to your original JSON file
with open(original_coco_file, 'r') as f:
    coco_data = json.load(f)

Про методы:
https://hasty.ai/docs/mp-wiki/augmentations/interpolation-methods

Про погодные эффекты
https://albumentations.ai/docs/examples/example_weather_transforms/

In [61]:
def get_augmentation_pipeline():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=30, p=0.2),
        A.OneOf([
            A.RandomBrightnessContrast(p=0.5),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5), # изменение цвета
            A.augmentations.transforms.GaussNoise(var_limit=(10,50),mean=0,p=0.5), # шум гаусса
            A.augmentations.transforms.CLAHE (clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p=0.5)
        ], p=0.3),
        A.Blur(blur_limit=3, p=0.1), # Размытие
        # A.augmentations.transforms.Equalize(),
        A.augmentations.geometric.transforms.Affine(p=0.1, shear={'x': (-30, 30), 'y': (-30, 30)}),
        A.OneOf([
            A.augmentations.transforms.ToGray(p=0.2), # оттенки серого
            A.RandomRain(brightness_coefficient=0.9, drop_width=1, blur_value=3, p=0.2), # добавляет дождь
            A.RandomSnow(brightness_coeff=2.5, snow_point_lower=0.3, snow_point_upper=0.5, p=0.2), #Добавляет снег
            A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.6, alpha_coef=0.1, p=0.2), # туман
            A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0.5, p=0.22), # солнечные блики
        ],p=0.3),
    ], bbox_params=A.BboxParams(format='coco', min_area=1024, min_visibility=0.3,))


In [49]:
# Create a new folder to save augmented images
output_image_folder = '/content/gdrive/MyDrive/urbanhack-train/augmented_images'
os.makedirs(output_image_folder, exist_ok=True)


In [50]:
# Initialize variables to track the augmented dataset
augmented_coco_data = {'images': [], 'annotations': [], 'categories': coco_data['categories']}


In [51]:
# Define the number of augmented images per original image
num_augmented_images = 10

In [62]:
# Perform augmentation and update annotations
for image_info in coco_data['images']:
    image_path = os.path.join(image_folder, image_info['file_name'])  # Replace with your image folder path
    image = cv2.imread(image_path)
    # image = np.array(image)

    bboxes = []
    for i in range(len(coco_data['annotations']))
          bboxes.append(coco_data['annotations'][])

    for _ in range(num_augmented_images):
        augmentation_pipeline = get_augmentation_pipeline()

        augmented = augmentation_pipeline(image=image, bboxes=coco_data['annotations'])
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']

        # Save augmented image
        output_image_path = os.path.join(output_image_folder, f"{image_info['file_name'][:-4]}_{np.random.randint(1, 1000)}.jpg")
        cv2.imwrite(output_image_path, augmented_image)

        # Update annotation information
        for bbox in augmented_bboxes:
            annotation = {
                'id': len(augmented_coco_data['annotations']) + 1,
                'image_id': image_info['id'],
                'category_id': bbox['category_id'],
                'segmentation': [],
                'area': bbox['area'],
                'bbox': bbox['bbox'],
                'iscrowd': 0,
                'attributes': {'occluded': False, 'rotation': 0.0},
            }
            augmented_coco_data['annotations'].append(annotation)

    # Copy the original image info to the augmented dataset
    augmented_coco_data['images'].append(image_info)

TypeError: ignored